# Libraries Import

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output
import tensorflow as tf
from tensorflow.keras import layers, models

## FIGURE GENERATOR

### LINES


In [ ]:
def line_vd(input,rec,n_frames,height=100, width=100):

    x1=np.random.randint(0, width-1)
    x2=np.random.randint(0, width-1)
    y1=np.random.randint(0, height-1)
    y2=np.random.randint(0, height-1)
    thickness=np.random.randint(1, 3)
    
    vd_ls=[]

    for _ in range(n_frames):
        
        image = np.zeros((height, width), dtype=np.uint8)

        x1+= np.random.randint(-int(width*0.02), int(width*0.02))
        x2+=np.random.randint(-int(width*0.06), int(width*0.06))
        y1+= np.random.randint(-int(height*0.02), int(height*0.02))
        y2+=np.random.randint(-int(height*0.06), int(height*0.06))
        
        cv2.line(image, (x1, y1), (x2, y2), color=255, thickness=thickness)
        vd_ls.append(image)

    input[rec]['vd_d']=np.array(vd_ls)
    input[rec]['label']=0

    return input

        

### CIRCLES


In [ ]:
def circle_vd(input,rec,n_frames,height=100, width=100):
    
    thickness=np.random.randint(1, 2)
    x= np.random.randint(int(0+width*0.3), int(width - width*0.3))
    y=np.random.randint(int(0+height*0.3), int(height-height*0.3))
    radio = np.random.randint(12, 17)
    vd_ls=[]

    for _ in range(n_frames):
        
        image = np.zeros((height, width), dtype=np.uint8)

        x+= np.random.randint(-int(width*0.03), int(width*0.03))
        y+= np.random.randint(-int(height*0.03), int(height*0.03))
        radio += np.random.randint(-int(height*0.01), int(height*0.02))
        
        cv2.circle(image, (x,y), radio, color=255, thickness=thickness)
        vd_ls.append(image)

    input[rec]['vd_d']=np.array(vd_ls)
    input[rec]['label']=1

    return input

    

# DATA GENERATOR

In [ ]:
height, wide = 100, 100
n_recs=30
min_frm,max_frm=5,10

input_d={'I'+ str(ii):{} for ii in range(n_recs)}

for R in input_d.keys():

    n_frames=np.random.randint(min_frm,max_frm)
    input_c=input_d.copy()
    n_rdm=np.random.rand()

    if n_rdm>=0.5:input_d=line_vd(input_c,R,n_frames,height, wide)
    else:input_d=circle_vd(input_c,R,n_frames,height, wide)


In [16]:
print('RECORD - LABEL - SHAPE')
for R in input_d.keys():
    print(R,input_d[R]['label'],np.shape(input_d[R]['vd_d']),type(input_d[R]['vd_d']))

RECORD - LABEL - SHAPE
I0 0 (8, 100, 100) <class 'numpy.ndarray'>
I1 1 (7, 100, 100) <class 'numpy.ndarray'>
I2 0 (7, 100, 100) <class 'numpy.ndarray'>
I3 0 (8, 100, 100) <class 'numpy.ndarray'>
I4 1 (9, 100, 100) <class 'numpy.ndarray'>
I5 0 (5, 100, 100) <class 'numpy.ndarray'>
I6 0 (7, 100, 100) <class 'numpy.ndarray'>
I7 0 (6, 100, 100) <class 'numpy.ndarray'>
I8 0 (8, 100, 100) <class 'numpy.ndarray'>
I9 1 (9, 100, 100) <class 'numpy.ndarray'>
I10 0 (6, 100, 100) <class 'numpy.ndarray'>
I11 1 (9, 100, 100) <class 'numpy.ndarray'>
I12 1 (6, 100, 100) <class 'numpy.ndarray'>
I13 1 (9, 100, 100) <class 'numpy.ndarray'>
I14 1 (9, 100, 100) <class 'numpy.ndarray'>
I15 1 (7, 100, 100) <class 'numpy.ndarray'>
I16 0 (8, 100, 100) <class 'numpy.ndarray'>
I17 1 (9, 100, 100) <class 'numpy.ndarray'>
I18 0 (8, 100, 100) <class 'numpy.ndarray'>
I19 1 (5, 100, 100) <class 'numpy.ndarray'>
I20 0 (8, 100, 100) <class 'numpy.ndarray'>
I21 0 (9, 100, 100) <class 'numpy.ndarray'>
I22 1 (8, 100, 100)

In [ ]:
video_check,rec_check=True,'I0'

if video_check:
    
    t_btw_frm=0.2

    for ii in range(len(input_d[rec_check]['vd_d'])):

        imagen = input_d[rec_check]['vd_d'][ii]

        plt.imshow(imagen, cmap='gray')
        plt.axis('off')
        plt.show()
        time.sleep(t_btw_frm)
        clear_output(wait=True)

# Neural network

### GENERATE TRAIN DATA

In [ ]:

max_frm_n=max([len(input_d[R]['vd_d']) for R in input_d.keys()])
x_train,y_train=[],[]

for R in input_d.keys():

    vd=input_d[R]['vd_d']
    vd = vd[:, :, :, np.newaxis]
    x_train.append(vd)
    y_train.append(input_d[R]['label'])

for i, video in enumerate(x_train):
    frames_actual = video.shape[0]
    if frames_actual < max_frm_n:
        
        padding = np.zeros((max_frm_n - frames_actual, height, wide, 1))
        x_train[i] = np.concatenate([video, padding], axis=0)
    elif frames_actual > max_frm_n:
        
        x_train[i] = video[:max_frm_n, :, :, :]

X_train = np.array(x_train)
Y_train = np.array(y_train)

#X_train, Y_train = X_train / 255.0, Y_train / 255.0

for ii in range(len(input_d.keys())): print(np.shape(X_train[ii]),Y_train[ii])


### GENERATE MODEL

In [ ]:

model = models.Sequential()

model.add(layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu'), input_shape=(max_frm_n, height, wide, 1)))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

model.add(layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

# Capa LSTM
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(16))

# Capa densa de salida
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

model.fit(X_train, Y_train, epochs=10, batch_size=32)


### MODEL PREDICTION

In [ ]:

verification_input_d={'LINE':{}, 'CIRCLE':{}}
n_frames=np.random.randint(min_frm,max_frm)

verification_input_c=verification_input_d.copy()
verification_input_d=line_vd(verification_input_c,'LINE',n_frames,height, wide)

verification_input_c=verification_input_d.copy()
verification_input_d=circle_vd(verification_input_c,'CIRCLE',n_frames,height, wide)



In [ ]:

print('RECORD - LABEL - SHAPE')
for R in verification_input_d.keys():
    print(R,verification_input_d[R]['label'],np.shape(verification_input_d[R]['vd_d']),type(verification_input_d[R]['vd_d']))


In [ ]:

t_btw_frm=0.2

for ii in range(len(verification_input_d['LINE']['vd_d'])):
    
    im_line = verification_input_d['LINE']['vd_d'][ii]
    im_circle = verification_input_d['CIRCLE']['vd_d'][ii]
    fig, axs = plt.subplots(1, 2)#, figsize=(10, 5))

    axs[0].imshow(im_line , cmap='gray')
    axs[0].axis('off')
    axs[1].imshow(im_circle, cmap='gray')
    axs[1].axis('off')

    plt.show()
    time.sleep(t_btw_frm)
    clear_output(wait=True)

In [ ]:
vd_line=verification_input_d['LINE']['vd_d']
vd_line = vd_line[:, :, :, np.newaxis]

if vd_line.shape[0] < max_frm_n:
    
    padding = np.zeros((max_frm_n - vd_line.shape[0],  height, wide, 1))
    vd_line = np.concatenate([vd_line, padding], axis=0)

vd_circle=verification_input_d['CIRCLE']['vd_d']
vd_circle = vd_circle[:, :, :, np.newaxis]

if vd_circle.shape[0] < max_frm_n:
    
    padding = np.zeros((max_frm_n - vd_circle.shape[0],  height, wide, 1))
    vd_circle = np.concatenate([vd_circle, padding], axis=0)

In [ ]:

pred_line = model.predict(np.expand_dims(vd_line, axis=0))
pred_line_r=np.round(pred_line[0][0], decimals=4)

pred_circle = model.predict(np.expand_dims(vd_circle, axis=0))
pred_circle_r=np.round(pred_circle[0][0], decimals=4)

print('VIDEO LÍNEA:')
if pred_line >=0.5: print(f'Es {pred_line_r*100} % CÍRCULO')
else:print(f'Es {(1-pred_line_r)*100} % LÍNEA')
print(' ')

print('VIDEO CÍRCULO:')
if pred_circle >=0.5: print(f'Es {(pred_circle_r)*100} % CÍRCULO')
else:print(f'Es {(1-pred_circle_r)*100} % LÍNEA')
